In [0]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [0]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/HLCV 19/Project/Project_HLCV_19/')
!ls

accuracies_without_batch_normalization.png
acgan
checkpoint
cnn_clf.pt
code
losses_without_batch_normalization.png
model.ckpt


In [0]:
def read_splits(folder,ext):
  #folder=path+folder
  files=os.listdir(folder)
  fileids=[]
  for f in files:
    if ext in f:
      fileids.append(folder+f)
  return fileids

# ft=read_splits('features_conv5_trained/','npy')
ft=read_splits('code/evaluate/features_unconv4_trained/','npy')

print(len(ft))

511


In [0]:
train=[]
for f in ft:
  train.append(np.load(f, allow_pickle = False))
  
train_features = train[0:454]  
train_features=np.asarray(train_features)
#train_features=np.reshape(train_features,(4540,-1))
train_features = train_features.reshape(-1, *train_features.shape[-3:])
train_features = np.swapaxes(train_features,1,3)

test_features = train[454:]  
test_features=np.asarray(test_features)
#test_features=np.reshape(test_features,(570,-1))
test_features = test_features.reshape(-1, *test_features.shape[-3:])
test_features = np.swapaxes(test_features,1,3)

#train_features=np.reshape(train_features,(len(ft),1024,-1))
del train
# testing with 1 feature
#train_features= np.load(ft[0], allow_pickle = False)
#train_features=np.reshape(train_features,(1024,-1))

#from sklearn.preprocessing import OneHotEncoder


print(train_features.shape,test_features.shape)

train_targets = np.load('code/evaluate/targets_conv5_trained/data_targets.npy', allow_pickle = False)
test_targets = np.load('code/evaluate/targets_conv5_trained/test_targets.npy', allow_pickle = False)

train_targets = np.squeeze(train_targets)
test_targets = np.squeeze(test_targets)

print(train_targets.shape,test_targets.shape)

In [0]:
num_classes = 2
y = torch.eye(num_classes)

test_targets = y[test_targets]
print(test_targets.shape)

torch.Size([570, 2])


In [0]:
train[0].shape

NameError: ignored

In [0]:
del train_features

In [0]:
# class _netG(nn.Module):
#   def __init__(self, ngpu, nz):
#       super(_netG, self).__init__()
#       self.ngpu = ngpu
#       self.nz = nz

#       # first linear layer
#       self.fc1 = nn.Linear(nz, 384)
#       # Transposed Convolution 2
#       self.tconv2 = nn.Sequential(
#           nn.ConvTranspose2d(384, 192, 4, 1, 0, bias=False),
#           nn.BatchNorm2d(192),
#           nn.ReLU(True),
#       )
#       # Transposed Convolution 3
#       self.tconv3 = nn.Sequential(
#           nn.ConvTranspose2d(192, 96, 4, 2, 1, bias=False),
#           nn.BatchNorm2d(96),
#           nn.ReLU(True),
#       )
#       # Transposed Convolution 4
#       self.tconv4 = nn.Sequential(
#           nn.ConvTranspose2d(96, 48, 4, 2, 1, bias=False),
#           nn.BatchNorm2d(48),
#           nn.ReLU(True),
#       )
#       # Transposed Convolution 4
#       self.tconv5 = nn.Sequential(
#           nn.ConvTranspose2d(48, 3, 4, 2, 1, bias=False),
#           nn.Tanh(),
#       )

#   def forward(self, input):
#       if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
#           input = input.view(-1, self.nz)
#           fc1 = nn.parallel.data_parallel(self.fc1, input, range(self.ngpu))
#           fc1 = fc1.view(-1, 384, 1, 1)
#           tconv2 = nn.parallel.data_parallel(self.tconv2, fc1, range(self.ngpu))
#           tconv3 = nn.parallel.data_parallel(self.tconv3, tconv2, range(self.ngpu))
#           tconv4 = nn.parallel.data_parallel(self.tconv4, tconv3, range(self.ngpu))
#           tconv5 = nn.parallel.data_parallel(self.tconv5, tconv4, range(self.ngpu))
#           output = tconv5
#       else:
#           input = input.view(-1, self.nz)
#           fc1 = self.fc1(input)
#           fc1 = fc1.view(-1, 384, 1, 1)
#           tconv2 = self.tconv2(fc1)
#           tconv3 = self.tconv3(tconv2)
#           tconv4 = self.tconv4(tconv3)
#           tconv5 = self.tconv5(tconv4)
#           output = tconv5
#       return output

In [0]:
# class _netD(nn.Module):
#     def __init__(self, ngpu, num_classes=10):
#         super(_netD, self).__init__()
#         self.ngpu = ngpu

#         # Convolution 1
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(1, 16, 3, 1, 1, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
# #             nn.Dropout(0.5, inplace=False),
#         )
#         # Convolution 2
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(16, 32, 3, 1, 1, bias=False),
#             nn.BatchNorm2d(32),
#             nn.LeakyReLU(0.2, inplace=True),
# #             nn.Dropout(0.5, inplace=False),
#         )
#         # Convolution 3
#         self.conv3 = nn.Sequential(
#             nn.Conv2d(32, 64, 3, 2, 1, bias=False),
#             nn.BatchNorm2d(64),
#             nn.LeakyReLU(0.2, inplace=True),
# #             nn.Dropout(0.5, inplace=False),
#         )
#         # Convolution 4
#         self.conv4 = nn.Sequential(
#             nn.Conv2d(64, 128, 3, 1, 1, bias=False),
#             nn.BatchNorm2d(128),
#             nn.LeakyReLU(0.2, inplace=True),
# #             nn.Dropout(0.5, inplace=False),
#         )
#        # Convolution 5
#         self.conv5 = nn.Sequential(
#             nn.Conv2d(128, 256, 3, 2, 1, bias=False),
#             nn.BatchNorm2d(256),
#             nn.LeakyReLU(0.2, inplace=True),
# #             nn.Dropout(0.5, inplace=False),
#         )
#        # Convolution 6
#         self.conv6 = nn.Sequential(
#             nn.Conv2d(256, 512, 3, 1, 1, bias=False),
#             nn.BatchNorm2d(512),
#             nn.LeakyReLU(0.2, inplace=True),
# #             nn.Dropout(0.5, inplace=False),
# #         )
#         # discriminator fc
#         self.fc_dis = nn.Linear(4*4*512, 1)
#         # aux-classifier fc
#         self.fc_aux = nn.Linear(4*4*512, num_classes)
#         # softmax and sigmoid
#         self.softmax = nn.Softmax()
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, input):
#         if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
#             conv1 = nn.parallel.data_parallel(self.conv1, input, range(self.ngpu))
#             conv2 = nn.parallel.data_parallel(self.conv2, conv1, range(self.ngpu))
#             conv3 = nn.parallel.data_parallel(self.conv3, conv2, range(self.ngpu))
#             conv4 = nn.parallel.data_parallel(self.conv4, conv3, range(self.ngpu))
#             conv5 = nn.parallel.data_parallel(self.conv5, conv4, range(self.ngpu))
#             conv6 = nn.parallel.data_parallel(self.conv6, conv5, range(self.ngpu))
#             flat6 = conv4.view(-1, 4*4*512)
#             fc_dis = nn.parallel.data_parallel(self.fc_dis, flat6, range(self.ngpu))
#             fc_aux = nn.parallel.data_parallel(self.fc_aux, flat6, range(self.ngpu))
#         else:
#             conv1 = self.conv1(input)
#             conv2 = self.conv2(conv1)
#             conv3 = self.conv3(conv2)
#             conv4 = self.conv4(conv3)
#             conv5 = self.conv5(conv4)
#             conv6 = self.conv6(conv5)
#             flat6 = conv4.view(-1, 4*4*512)
#             fc_dis = self.fc_dis(flat6)
#             fc_aux = self.fc_aux(flat6)
#         classes = self.softmax(fc_aux)
#         realfake = self.sigmoid(fc_dis).view(-1, 1).squeeze(1)
#         return realfake, classes

In [0]:
# # custom weights initialization called on netG and netD
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         m.weight.data.normal_(0.0, 0.02)
#     elif classname.find('BatchNorm') != -1:
#         m.weight.data.normal_(1.0, 0.02)
#         m.bias.data.fill_(0)

# compute the current classification accuracy
def compute_acc(preds, labels):
    correct = 0
    preds_ = preds.data.max(1)[1]
    correct = preds_.eq(labels.data).cpu().sum()
    acc = float(correct) / float(len(labels.data)) * 100.0
    return acc

In [0]:
# architecture from Xian et al 2018

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class MLP_CRITIC(nn.Module):
    def __init__(self, ngpu, num_classes=10): 
        super(MLP_CRITIC, self).__init__()
        self.ngpu = ngpu
        self.fc1 = nn.Linear(resSize, ndh)
        #self.fc2 = nn.Linear(ndh, ndh)
        self.lrelu = nn.LeakyReLU(0.2, True)
        # discriminator fc
        self.fc_dis = nn.Linear(ndh, 1)
        # aux-classifier fc
        self.fc_aux = nn.Linear(ndh, num_classes)
        # softmax and sigmoid
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()

        self.apply(weights_init)

    def forward(self, x):
        h = self.lrelu(self.fc1(x))
#         h = self.fc2(h)
        fc_dis = self.fc_dis(h)
        fc_aux = self.fc_aux(h)
        classes = self.softmax(fc_aux)
        realfake = self.sigmoid(fc_dis).view(-1, 1).squeeze(1)
        return realfake, classes


class MLP_G(nn.Module):
    def __init__(self, ngpu,nz):
        super(MLP_G, self).__init__()
        self.ngpu = ngpu
        self.nz = nz
        
        self.fc1 = nn.Linear(nz, ngh)
        self.fc2 = nn.Linear(ngh, resSize)
        self.lrelu = nn.LeakyReLU(0.2, True)
        #self.prelu = nn.PReLU()
        self.relu = nn.ReLU(True)

        self.apply(weights_init)

    def forward(self, noise):
        input = noise.view(-1, self.nz)
        h = self.lrelu(self.fc1(input))
        h = self.relu(self.fc2(h))
        return h


In [0]:
toy_labels = np.random.randint(0, num_classes, 1024)



In [0]:
#--------------------------------
# Device configuration
#--------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

#--------------------------------
# Hyper-parameters
#--------------------------------
num_classes = 2
resSize = train_features.shape[1] #size of visual features
# attSize = 1024 #size of semantic features
nz = 312 #size of the latent z vector 
ngh = 4096 #size of the hidden units in generator
ndh = 1024 #size of the hidden units in discriminator
# hidden_size = [128, 512, 512, 512, 512, 512]
num_epochs = 5
batch_size = 200
learning_rate = 2e-3
learning_rate_decay = 0.95
reg=0.001
beta_1=0.5

num_training= train_features.shape[0]
num_validation = test_features.shape[0]
# print(hidden_size)

train_data = []
val_data=[]
for i in range(num_training):
   train_data.append([train_features[i], train_targets[i]])
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

for i in range(num_validation):
   val_data.append([test_features[i], test_targets[i]])
val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)


Using device: cuda


In [0]:
# ngpu=1
# netG = MLP_G(ngpu, nz)
# netG.apply(weights_init)
# print(netG)

# netD = MLP_CRITIC(ngpu, num_classes)
# netD.apply(weights_init)

# print(netD)

# # loss functions
# dis_criterion = nn.BCELoss()
# aux_criterion = nn.NLLLoss()

# # tensor placeholders
# x = torch.FloatTensor(batch_size, 3, input_size, input_size)
# noise = torch.FloatTensor(batch_size, nz, 1, 1)
# eval_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
# dis_label = torch.FloatTensor(batch_size)
# aux_label = torch.LongTensor(batch_size)
# real_label = 1
# fake_label = 0

# # if using cuda
# if device=='cuda':
#     netD.cuda()
#     netG.cuda()
#     dis_criterion.cuda()
#     aux_criterion.cuda()
#     x, dis_label, aux_label = x.cuda(), dis_label.cuda(), aux_label.cuda()
#     noise, eval_noise = noise.cuda(), eval_noise.cuda()

# # define variables
# x = Variable(x)
# noise = Variable(noise)
# eval_noise = Variable(eval_noise)
# dis_label = Variable(dis_label)
# aux_label = Variable(aux_label)
# # noise for evaluation
# eval_noise_ = np.random.normal(0, 1, (batch_size, nz))
# eval_label = np.random.randint(0, num_classes, batch_size)
# eval_onehot = np.zeros((batch_size, num_classes))
# eval_onehot[np.arange(batch_size), eval_label] = 1
# eval_noise_[np.arange(batch_size), :num_classes] = eval_onehot[np.arange(batch_size)]
# eval_noise_ = (torch.from_numpy(eval_noise_))
# eval_noise.data.copy_(eval_noise_.view(batch_size, nz, 1, 1))

# # setup optimizer
# optimizerD = optim.Adam(netD.parameters(), lr=learning_rate, betas=(beta_1, 0.999))
# optimizerG = optim.Adam(netG.parameters(), lr=learning_rate, betas=(beta_1, 0.999))

In [0]:
ngpu=1
netG = MLP_G(ngpu, nz)
netG.apply(weights_init)
print(netG)

netD = MLP_CRITIC(ngpu, num_classes)
netD.apply(weights_init)

print(netD)

# loss functions
dis_criterion = nn.BCELoss()
aux_criterion = nn.NLLLoss()

# tensor placeholders
x = torch.FloatTensor(batch_size, resSize)
noise = torch.FloatTensor(batch_size, nz)
eval_noise = torch.FloatTensor(batch_size, nz).normal_(0, 1)
dis_label = torch.FloatTensor(batch_size)
print(dis_label.shape)
aux_label = torch.LongTensor(batch_size)
print(aux_label.shape)
real_label = 1
fake_label = 0

# if using cuda
if device=='cuda':
    netD.cuda()
    netG.cuda()
    dis_criterion.cuda()
    aux_criterion.cuda()
    x, dis_label, aux_label = x.cuda(), dis_label.cuda(), aux_label.cuda()
    noise, eval_noise = noise.cuda(), eval_noise.cuda()

# define variables
x = Variable(x)
noise = Variable(noise)
eval_noise = Variable(eval_noise)
dis_label = Variable(dis_label)
aux_label = Variable(aux_label)
# noise for evaluation
eval_noise_ = np.random.normal(0, 1, (batch_size, nz))
eval_label = np.random.randint(0, num_classes, batch_size)
eval_onehot = np.zeros((batch_size, num_classes))
eval_onehot[np.arange(batch_size), eval_label] = 1
eval_noise_[np.arange(batch_size), :num_classes] = eval_onehot[np.arange(batch_size)]
eval_noise_ = (torch.from_numpy(eval_noise_))
eval_noise.data.copy_(eval_noise_.view(batch_size, nz))

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=learning_rate, betas=(beta_1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=learning_rate, betas=(beta_1, 0.999))

MLP_G(
  (fc1): Linear(in_features=312, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=230400, bias=True)
  (lrelu): LeakyReLU(negative_slope=0.2, inplace)
  (relu): ReLU(inplace)
)
MLP_CRITIC(
  (fc1): Linear(in_features=230400, out_features=1024, bias=True)
  (lrelu): LeakyReLU(negative_slope=0.2, inplace)
  (fc_dis): Linear(in_features=1024, out_features=1, bias=True)
  (fc_aux): Linear(in_features=1024, out_features=2, bias=True)
  (softmax): Softmax()
  (sigmoid): Sigmoid()
)
torch.Size([2])
torch.Size([2])


In [0]:
# avg_loss_D = 0.0
# avg_loss_G = 0.0
# avg_loss_A = 0.0
# for epoch in range(num_epochs):
#     for i, data in enumerate(train_loader):
#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
#         # train with real
#         with torch.no_grad():
#           real_cpu, label = data
#           batch_size = real_cpu.size(0)
#           x.resize_(real_cpu.size()).copy_(real_cpu)
#         dis_label.data.resize_(batch_size).fill_(real_label)
#         aux_label.data.resize_(batch_size).copy_(labe0..
#                                                  l)
#         dis_output, aux_output = netD(x)

#         dis_errD_real = dis_criterion(dis_output, dis_label)
#         aux_errD_real = aux_criterion(aux_output, aux_label)
#         errD_real = dis_errD_real + aux_errD_real
#         errD_real.backward()
#         D_x = dis_output.data.mean()

#         # compute the current classification accuracy
#         accuracy = compute_acc(aux_output, aux_label)

#         # train with fake
#         noise.data.resize_(batch_size, nz, 1, 1).normal_(0, 1)
#         label = np.random.randint(0, num_classes, batch_size)
#         noise_ = np.random.normal(0, 1, (batch_size, nz))
#         class_onehot = np.zeros((batch_size, num_classes))
#         class_onehot[np.arange(batch_size), label] = 1
#         noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]
#         noise_ = (torch.from_numpy(noise_))
#         noise.data.copy_(noise_.view(batch_size, nz, 1, 1))
#         aux_label.data.resize_(batch_size).copy_(torch.from_numpy(label))

#         fake = netG(noise)
#         dis_label.data.fill_(fake_label)
#         dis_output, aux_output = netD(fake.detach())
#         dis_errD_fake = dis_criterion(dis_output, dis_label)
#         aux_errD_fake = aux_criterion(aux_output, aux_label)
#         errD_fake = dis_errD_fake + aux_errD_fake
#         errD_fake.backward()
#         D_G_z1 = dis_output.data.mean()
#         errD = errD_real + errD_fake
#         optimizerD.step()

#         ############################
#         # (2) Update G network: maximize log(D(G(z)))
#         ###########################
#         netG.zero_grad()
#         dis_label.data.fill_(real_label)  # fake labels are real for generator cost
#         dis_output, aux_output = netD(fake)
#         dis_errG = dis_criterion(dis_output, dis_label)
#         aux_errG = aux_criterion(aux_output, aux_label)
#         errG = dis_errG + aux_errG
#         errG.backward()
#         D_G_z2 = dis_output.data.mean()
#         optimizerG.step()

#         # compute the average loss
#         curr_iter = epoch * len(train_loader) + i
#         all_loss_G = avg_loss_G * curr_iter
#         all_loss_D = avg_loss_D * curr_iter
#         all_loss_A = avg_loss_A * curr_iter
#         all_loss_G += errG.item()
#         all_loss_D += errD.item()
#         all_loss_A += accuracy
#         avg_loss_G = all_loss_G / (curr_iter + 1)
#         avg_loss_D = all_loss_D / (curr_iter + 1)
#         avg_loss_A = all_loss_A / (curr_iter + 1)

#         print('[%d/%d][%d/%d] Loss_D: %.4f (%.4f) Loss_G: %.4f (%.4f) D(x): %.4f D(G(z)): %.4f / %.4f Acc: %.4f (%.4f)'
#               % (epoch, num_epochs, i, len(train_loader),
#                  errD.item(), avg_loss_D, errG.item(), avg_loss_G, D_x, D_G_z1, D_G_z2, accuracy, avg_loss_A))
#         if i % 100 == 0:
#             vutils.save_image(real_cpu, 'real_samples.png')
#             print('Label for eval = {}'.format(eval_label))
#             fake = netG(eval_noise)
#             vutils.save_image(fake.data,'fake_samples_epoch_%03d.png' % ( epoch))
        
            

#     # do checkpointing
#     torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (opt.outf, epoch))
#     torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (opt.outf, epoch))

In [0]:
avg_loss_D = 0.0
avg_loss_G = 0.0
avg_loss_A = 0.0
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        with torch.no_grad():
          real_x, label= data
          print(real_x.size())
          batch_size = real_x.size(0)
          x.resize_(real_x.size()).copy_(real_x)
        dis_label.data.resize_(batch_size).fill_(real_label)
        aux_label.data.resize_(batch_size).copy_(label)
        dis_output, aux_output = netD(x)
        print(dis_output.size())

        dis_errD_real = dis_criterion(dis_output, dis_label)
        aux_errD_real = aux_criterion(aux_output, aux_label)
        errD_real = dis_errD_real + aux_errD_real
        errD_real.backward()
        D_x = dis_output.data.mean()

        # compute the current classification accuracy
        accuracy = compute_acc(aux_output, aux_label)

        # train with fake
        noise.data.resize_(batch_size, nz).normal_(0, 1)
        label = np.random.randint(0, num_classes, batch_size)
        noise_ = np.random.normal(0, 1, (batch_size, nz))
        class_onehot = np.zeros((batch_size, num_classes))
        class_onehot[np.arange(batch_size), label] = 1
        noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]
        noise_ = (torch.from_numpy(noise_))
        noise.data.copy_(noise_.view(batch_size, nz))
        aux_label.data.resize_(batch_size).copy_(torch.from_numpy(label))

        fake = netG(noise)
        dis_label.data.fill_(fake_label)
        dis_output, aux_output = netD(fake.detach())
        dis_errD_fake = dis_criterion(dis_output, dis_label)
        aux_errD_fake = aux_criterion(aux_output, aux_label)
        errD_fake = dis_errD_fake + aux_errD_fake
        errD_fake.backward()
        D_G_z1 = dis_output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        dis_label.data.fill_(real_label)  # fake labels are real for generator cost
        dis_output, aux_output = netD(fake)
        dis_errG = dis_criterion(dis_output, dis_label)
        aux_errG = aux_criterion(aux_output, aux_label)
        errG = dis_errG + aux_errG
        errG.backward()
        D_G_z2 = dis_output.data.mean()
        optimizerG.step()

        # compute the average loss
        curr_iter = epoch * len(train_loader) + i
        all_loss_G = avg_loss_G * curr_iter
        all_loss_D = avg_loss_D * curr_iter
        all_loss_A = avg_loss_A * curr_iter
        all_loss_G += errG.item()
        all_loss_D += errD.item()
        all_loss_A += accuracy
        avg_loss_G = all_loss_G / (curr_iter + 1)
        avg_loss_D = all_loss_D / (curr_iter + 1)
        avg_loss_A = all_loss_A / (curr_iter + 1)

        print('[%d/%d][%d/%d] Loss_D: %.4f (%.4f) Loss_G: %.4f (%.4f) D(x): %.4f D(G(z)): %.4f / %.4f Acc: %.4f (%.4f)'
              % (epoch, num_epochs, i, len(train_loader),
                 errD.item(), avg_loss_D, errG.item(), avg_loss_G, D_x, D_G_z1, D_G_z2, accuracy, avg_loss_A))
        if i % 100 == 0:
            print('Label for eval = {}'.format(eval_label))
            fake = netG(eval_noise)
        
            

    # do checkpointing
#     torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (opt.outf, epoch))
#     torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (opt.outf, epoch))

torch.Size([2, 230400])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([2])


In [0]:
class _netD(nn.Module):
    def __init__(self, ngpu, num_classes=2):
        super(_netD, self).__init__()
        self.ngpu = ngpu

        # Convolution 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(1024, 512, 3, 1, 1, bias=False),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.LeakyReLU(0.2, inplace=True),
#             nn.Dropout(0.5, inplace=False),
        )
        # Convolution 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(512, 512, 3, 1, 1, bias=False),
            #nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
#             nn.Dropout(0.5, inplace=False),
        )

        # discriminator fc
        #self.fc_dis = nn.Linear(7*7*512, 1)
        # aux-classifier fc
        self.fc_aux1 = nn.Linear(7*7*512, 4*4*256)
        
        self.fc_aux2 = nn.Linear(4*4*256, num_classes)
        
        # softmax and sigmoid
        self.softmax = nn.Softmax()
        #self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            conv1 = nn.parallel.data_parallel(self.conv1, input, range(self.ngpu))
            conv2 = nn.parallel.data_parallel(self.conv2, conv1, range(self.ngpu))
            
            flat6 = conv2.view(-1, 7*7*512)
            #fc_dis = nn.parallel.data_parallel(self.fc_dis, flat6, range(self.ngpu))
            fc_aux = nn.parallel.data_parallel(self.fc_aux, flat6, range(self.ngpu))
        else:
            conv1 = self.conv1(input)
            conv2 = self.conv2(conv1)
            
            flat6 = conv2.view(-1, 7*7*512)
            #fc_dis = self.fc_dis(flat6)
            fc_aux1 = self.fc_aux1(flat6)
            
            fc_aux2 = self.fc_aux2(fc_aux1)
            
        classes = self.softmax(fc_aux2)
        #realfake = self.sigmoid(fc_dis).view(-1, 1).squeeze(1)
        return classes

In [0]:
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 1e-3)
        m.bias.data.fill_(0.)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

        
model = _netD(1).to(device)
# Q2.a - Initialize the model with correct batch norm layer

model.apply(weights_init)
# Print the model
print(model)

Using device: cuda
_netD(
  (conv1): Sequential(
    (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): LeakyReLU(negative_slope=0.2, inplace)
  )
  (conv2): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (fc_aux1): Linear(in_features=25088, out_features=4096, bias=True)
  (fc_aux2): Linear(in_features=4096, out_features=2, bias=True)
  (softmax): Softmax()
)


In [0]:
#--------------------------------
# Hyper-parameters
#--------------------------------

num_classes = 2

num_epochs = 20
batch_size = 200
learning_rate = 2e-3
learning_rate_decay = 0.95
reg=0.001

rotation_=5
translation_=0.1
hue_=0.05
saturation_=0.05





# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)
# *****START OF MODIFIED CODE (SELF-ADDED)*****
train_losses = []
train_loss = 0
val_losses = []
val_loss = 0
train_accuracies = []
val_accuracies = []
early_stopping = 0
batch_counter = 0
# *****END OF MODIFIED CODE (SELF-ADDED)*****
# Train the model
lr = learning_rate
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader): #245*200 training samples ? 200=batch_size?
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        #print(labels)
        # Forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # *****START OF MODIFIED CODE (SELF-ADDED)*****
        batch_counter = batch_counter + 1
        train_loss += loss.item()
        # *****END OF MODIFIED CODE (SELF-ADDED)*****
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # *****START OF MODIFIED CODE (SELF-ADDED)*****
    train_losses.append(train_loss/batch_counter)
    batch_counter = 0
    train_loss = 0
    # *****END OF MODIFIED CODE (SELF-ADDED)*****
    # Code to update the lr
    lr *= learning_rate_decay
    update_lr(optimizer, lr)
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            print(outputs.shape)
            print(labels.shape)
            _, labels = torch.max(labels.data, 1)
            _, predicted = torch.max(outputs.data, 1)
            print(predicted.shape)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # *****START OF MODIFIED CODE (SELF-ADDED)*****
            batch_counter = batch_counter + 1
            val_loss += criterion(outputs, labels).item()
            # *****END OF MODIFIED CODE (SELF-ADDED)*****
            
        # *****START OF MODIFIED CODE (SELF-ADDED)*****
        val_losses.append(val_loss/batch_counter)
        batch_counter = 0
        val_loss = 0
        # *****END OF MODIFIED CODE (SELF-ADDED)*****

        print('Validataion accuracy is: {} %'.format(100 * correct / total))
        #################################################################################
        # TODO: Q2.b Implement the early stopping mechanism to save the model which has #
        # acheieved the best validation accuracy so-far.                                #
        #################################################################################
        best_model = None
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        val_acc = 100 * correct / total
        if epoch == 0:
          val_acc_max = 0
        if val_acc > val_acc_max:
            best_model = val_acc
            print('Validation accuracy increased ({} --> {}).  Saving model ...'.format(val_acc_max, val_acc))
            torch.save(model.state_dict(), 'model.ckpt')
            val_acc_max = val_acc
            early_stopping = epoch
        else:
            print("No improvement in validation accuracy")
        
        val_accuracies.append(val_acc)
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        # *****START OF MODIFIED CODE (SELF-ADDED)*****
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print('Train accuracy is: {} %'.format(100 * correct / total))
        train_accuracies.append(100 * correct / total)      
        # *****END OF MODIFIED CODE (SELF-ADDED)*****

    model.train()

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
model.eval()
# *****START OF MODIFIED CODE (SELF-ADDED)*****
from matplotlib.ticker import MaxNLocator
plt.figure().gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.plot(train_losses, label='Training loss', marker='o')
plt.plot(val_losses, label='Validation loss', marker='o')
plt.legend()
plt.xlabel('Epoch Number')
plt.savefig('losses_without_batch_normalization.png')
plt.show()

plt.figure().gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.plot(train_accuracies, label='Train Accuracies', marker='o')
plt.plot(val_accuracies, label='Validation Accuracies', marker='o')
plt.axvline(x = early_stopping, color='k', linestyle='--', label='Early Stopping')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylim((0, 100))
plt.savefig('accuracies_without_batch_normalization.png')
plt.show()
# *****END OF MODIFIED CODE (SELF-ADDED)*****
# *****START OF MODIFIED CODE (SELF-ADDED)*****
# from google.colab import drive
# drive.mount('/content/gdrive')
# import json
# path = '/content/gdrive/My Drive'
# *****END OF MODIFIED CODE (SELF-ADDED)*****

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/20], Step [10/23], Loss: 0.9583
Epoch [1/20], Step [20/23], Loss: 0.8733
torch.Size([200, 2])
torch.Size([200, 2])
torch.Size([200])
torch.Size([200, 2])
torch.Size([200, 2])
torch.Size([200])
torch.Size([170, 2])
torch.Size([170, 2])
torch.Size([170])
Validataion accuracy is: 36.49122807017544 %
Validation accuracy increased (0 --> 36.49122807017544).  Saving model ...
Train accuracy is: 40.26431718061674 %
Epoch [2/20], Step [10/23], Loss: 0.9233
Epoch [2/20], Step [20/23], Loss: 0.7533
torch.Size([200, 2])
torch.Size([200, 2])
torch.Size([200])
torch.Size([200, 2])
torch.Size([200, 2])
torch.Size([200])
torch.Size([170, 2])
torch.Size([170, 2])
torch.Size([170])
Validataion accuracy is: 63.50877192982456 %
Validation accuracy increased (36.49122807017544 --> 63.50877192982456).  Saving model ...
Train accuracy is: 59.73568281938326 %
Epoch [3/20], Step [10/23], Loss: 0.7433
Epoch [3/20], Step [20/23], Loss: 0.6933
torch.Size([200, 2])
torch.Size([200, 2])
torch.Size([200])


KeyboardInterrupt: ignored